In [2]:
import pandas as pd
import numpy as np
import shutil as sh
from tqdm import tqdm

In [3]:
import os
# path to UPENN Dataset
path = 'E:/ML projects/gliobastoma-mgmt/'
path_modalities=path+ 'NifTI-files/'
path_structural_images=path_modalities+'images_structural/'
path_automated_seg=path_modalities+'automated_segm/'
path_seg=path_modalities+'images_segm/'

In [ ]:
df = pd.read_csv(
    'E:/ML projects/gliobastoma-mgmt/Codes/Arbitrary/UPENN-GBM_data_availability.csv')
df.columns = [c.replace(' ', '_') for c in df.columns]

df1 = pd.DataFrame({
    "id": [],
    "mgmt": [],
})

for i in tqdm(range(len(df))):
    if df.MGMT[i] == "available" and (df.Automatic_Tumor_Segmentation[i] == "available" or df.Corrected_Tumor_Segmentation[i]== "available") and df.Structural_imaging[i] == "available" and df.Overall_Survival[i] =="available" :
        df2 = pd.DataFrame({'id': [df.ID[i]],
                           'mgmt': ["available"]},
                           )
        df1 = df1.append(df2)

In [ ]:
df3=pd.read_csv('E:/ML projects/gliobastoma-mgmt/Codes/Arbitrary/UPENN-GBM_clinical_info_v1.1.csv')

df4=pd.DataFrame({"id": [], "mgmt":[], "age":[], "gender":[], "survival":[]})

from tqdm import tqdm
arr=df1.id.to_numpy()
print("Total patients: ",len(arr))
count=0

for i in tqdm(range(len(df3))):
   
    if df3.ID[i] in arr:
        
        l="0"
        
        if df3.MGMT[i] == 'Unmethylated':
            pass
        else:
            l="1"
        # print(df3.MGMT[i])
        df5 = pd.DataFrame({'id': [df3.ID[i]],
                           'mgmt': [l],
                           'age': [df3.Age_at_scan_years[i]],
                           'gender': [df3.Gender[i]],
                           "survival": [df3.Survival_from_surgery_days[i]]
                            })
        df4= df4.append(df5)

df4.to_csv('upenn_data.csv')

In [4]:
df6=pd.read_csv(path+'Codes/upenn_data.csv')

m_neg = path + 'Data/MGMT_negative/'
m_pos = path + 'Data/MGMT_positive/'

pos = 0
neg = 0
for i in tqdm(range(len(df6))):
    path1 = path_structural_images+df6.id[i]
    path_segm = path_seg+df6.id[i]+'_segm.nii.gz'
    path_auto_segm = path_automated_seg+df6.id[i]+'_automated_approx_segm.nii.gz'
    rename_segm = df6.id[i]+'_seg.nii.gz'

    if df6.mgmt[i]==0 and neg<3: # only 5 negatives
        path2= m_neg+df6.id[i]+'/'
        seg_path = path2+rename_segm
        sh.copytree(path1,path2)
        if(os.path.exists(path_segm)):
            sh.copy(path_segm,seg_path)
        elif(os.path.exists(path_auto_segm)):
            sh.copy(path_auto_segm,seg_path)
        neg+=1
    elif df6.mgmt[i]==1 and pos<3: # only 5 positives
        path2=m_pos+df6.id[i]+'/'
        seg_path = path2+rename_segm
        sh.copytree(path1,path2)
        if(os.path.exists(path_segm)):
            sh.copy(path_segm,seg_path)
        elif(os.path.exists(path_auto_segm)):
            sh.copy(path_auto_segm,seg_path)
        else:
            print("No segmentation found for ",df6.id[i])
        pos+=1

100%|██████████| 213/213 [00:02<00:00, 73.01it/s]
